In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Re-load the dataset
df_new = pd.read_csv('../data/transform/transformed_data.csv')

In [2]:
cr = ['match_api_id', 'season', 'stage']
df_numerical = df_new.drop(cr, axis=1)
df_numerical.to_csv('../data/transform/df_numerical.csv')
df_numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          3040 non-null   object 
 1   away_team                     3040 non-null   int64  
 2   home_team                     3040 non-null   int64  
 3   home_team_goal                3040 non-null   int64  
 4   away_team_goal                3040 non-null   int64  
 5   result_match                  3040 non-null   int64  
 6   away_possession               3040 non-null   int64  
 7   away_shoton                   3040 non-null   int64  
 8   home_possession               3040 non-null   int64  
 9   home_shoton                   3040 non-null   int64  
 10  player_rating_home_player_1   3040 non-null   int64  
 11  player_rating_home_player_2   3040 non-null   int64  
 12  player_rating_home_player_3   3040 non-null   int64  
 13  pla

In [3]:
def remove_highly_correlated_features(df, threshold=0.9, cols_to_remove=['date', 'away_team', 'home_team','home_team_goal',]):
    """
    Removes features from the dataframe that have a correlation coefficient
    higher than the specified threshold.

    Parameters:
    df (pd.DataFrame): Dataframe with numerical features.
    threshold (float): Threshold for high correlation.

    Returns:
    tuple: (set of features to remove, reduced dataframe shape)
    """
    # Calculate the correlation matrix
    correlation_matrix = df.drop(cols_to_remove, axis=1).corr()

    # Find feature pairs with correlation greater than the threshold
    # Interested in absolute value of correlation coefficient
    highly_correlated_pairs = correlation_matrix.abs().unstack().sort_values(ascending=False)
    highly_correlated_pairs = highly_correlated_pairs[highly_correlated_pairs >= threshold]
    highly_correlated_pairs = highly_correlated_pairs[highly_correlated_pairs < 1]  # Remove self-correlation

    # Since correlation is symmetric, we'll take one of each pair
    # We will only take the first of the pair to remove.
    features_to_remove = set([pair[0] for pair in highly_correlated_pairs.index])

    print(features_to_remove)
    df_ = df.drop(columns=features_to_remove)
    print(df_.shape)
    
    return df_

# data_reduced = remove_highly_correlated_features(df_numerical)

In [4]:
# Splitting the original dataset
X = df_numerical.drop(['result_match', 'date', 'away_team', 'away_team_goal', 'home_team','home_team_goal', 'away_possession', 'home_shoton', 'home_possession', 'away_shoton', ], axis=1)
y = df_numerical['result_match']
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBClassifier
xgb_full = XGBClassifier(random_state=42)

# Train the model on the full feature set
xgb_full.fit(X_train_full, y_train)

# Predictions and evaluation on the full feature set
y_pred_full = xgb_full.predict(X_test_full)
accuracy_full = accuracy_score(y_test, y_pred_full)

accuracy_full

0.47368421052631576

In [5]:
# Feature engineering: Aggregated player ratings for home and away teams
player_rating_columns_home = [col for col in X.columns if 'home_player' in col]
player_rating_columns_away = [col for col in X.columns if 'away_player' in col]

df_numerical['avg_player_rating_home'] = X[player_rating_columns_home].mean(axis=1)
df_numerical['avg_player_rating_away'] = X[player_rating_columns_away].mean(axis=1)

In [6]:
# Boxplot for 'avg_player_rating_home' vs 'result_match'
plt.figure(figsize=(10, 6))
sns.boxplot(x='result_match', y='avg_player_rating_away', data=df_numerical)
plt.title('Boxplot of Average Home Player Rating by Match Result')
plt.show()

In [7]:
# Clear negative correaltion between result_match and avg_palyer_rating_away

In [8]:
# List of player positions for which we have ratings
player_positions = range(1, 12)

# Calculate the difference from the average for each player
for position in player_positions:
    home_player_col = f'player_rating_home_player_{position}'
    away_player_col = f'player_rating_away_player_{position}'
    
    # Create new features for home and away players
    df_numerical[f'diff_player_{position}'] = df_numerical[home_player_col] - df_numerical[away_player_col]
    # data_reduced.drop([home_player_col, away_player_col], axis=1, inplace=True)

# Display the dataframe to confirm the new features
df_numerical.filter(like='diff_player').columns

Index(['diff_player_1', 'diff_player_2', 'diff_player_3', 'diff_player_4',
       'diff_player_5', 'diff_player_6', 'diff_player_7', 'diff_player_8',
       'diff_player_9', 'diff_player_10', 'diff_player_11'],
      dtype='object')

In [ ]:
def calculate_rolling_averages(rolling_window_size, df, stat_columns):
    """
    Calculate rolling averages for various statistics without including the current match.

    Args:
    - rolling_window_size (int): Number of previous matches to include in the rolling average.
    - df (pd.DataFrame): The DataFrame containing the match data.
    - stat_columns (dict): A dictionary with keys for each stat category (e.g., 'goals', 'possession', 'shots_on'),
                           and values being another dictionary with 'home' and 'away' keys mapping to the respective columns.

    Returns:
    - pd.DataFrame: The DataFrame with new rolling average columns added.
    """

    # Iterate through each stat category
    for stat, columns in stat_columns.items():
        # Define new feature names for rolling features
        home_feature_name = f'home_{stat}_rolling'
        away_feature_name = f'away_{stat}_rolling'

        # Calculate rolling averages for home and away statistics
        for team_type in ['home', 'away']:
            # Sort by team and date
            df.sort_values(by=[f'{team_type}_team', 'date'], ascending=True, inplace=True)
            
            # Calculate rolling averages for the given stat
            df[home_feature_name if team_type == 'home' else away_feature_name] = (
                df.groupby(f'{team_type}_team')[columns[team_type]]
                .apply(lambda x: x.shift().rolling(window=rolling_window_size).mean())
                .reset_index(level=0, drop=True)
            )
            
            # Calculate the means for rolling features
            rolling_means = df.groupby(f'{team_type}_team')[home_feature_name if team_type == 'home' else away_feature_name].transform('mean')
            
            # Fill NaN values in rolling features with the team-specific means
            df[home_feature_name if team_type == 'home' else away_feature_name] = (
                df[home_feature_name if team_type == 'home' else away_feature_name].fillna(rolling_means)
            )

    return df

In [ ]:
# Define a range of window sizes to test
window_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15]

# Define your statistical categories and their corresponding columns
stat_columns = {
    'goals': {
        'home': 'home_team_goal',
        'away': 'away_team_goal'
    },
    'possession': {
        'home': 'home_possession',
        'away': 'away_possession'
    },
    'shots_on': {
        'home': 'home_shoton',
        'away': 'away_shoton'
    }
}

# Placeholder for best score and corresponding window size
best_score = -float('inf')
best_window_size = None

# Iterate over window sizes to find the best one
for window_size in window_sizes:
    # Calculate rolling averages using the current window size
    df_with_rolling = calculate_rolling_averages(window_size, df_numerical.copy(), stat_columns)

    df_with_rolling = df_with_rolling.drop(['date', 'away_team', 'away_team_goal', 'home_team','home_team_goal', 'away_possession', 'home_shoton', 'home_possession', 'away_shoton'], axis=1)
    
    # Extract features and target for model training
    X = df_with_rolling.drop(['result_match'], axis=1)
    y = df_with_rolling['result_match']
    
    # Define your model
    model = XGBClassifier(random_state=42)  # Use your actual model here
    
    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=2, scoring='accuracy')  # Change scoring method as needed
    
    # Calculate the mean score
    mean_score = scores.mean()
    
    # Update best score and window size if current score is better
    if mean_score > best_score:
        best_score = mean_score
        best_window_size = window_size

# best_window_size now holds the size of the window that gave the best performance
print(f"Best window size: {best_window_size} with score: {best_score}")

In [ ]:
df_with_rolling = calculate_rolling_averages(best_window_size, df=df_numerical.copy(), stat_columns=stat_columns)
df_with_rolling = df_with_rolling.drop(['date', 'away_team', 'away_team_goal', 'home_team','home_team_goal', 'away_possession', 'home_shoton', 'home_possession', 'away_shoton'], axis=1)

In [ ]:
# Splitting the original dataset
X = df_with_rolling.drop('result_match', axis=1)
y = df_with_rolling['result_match']
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBClassifier
xgb_full = XGBClassifier(random_state=42)

# Train the model on the full feature set
xgb_full.fit(X_train_full, y_train)

# Predictions and evaluation on the full feature set
y_pred_full = xgb_full.predict(X_test_full)
accuracy_full = accuracy_score(y_test, y_pred_full)

accuracy_full

In [ ]:
df_with_rolling.info()

In [18]:
# Splitting the original dataset
X = df.drop('result_match', axis=1)
y = df['result_match']
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

from sklearn.model_selection import cross_val_score

# Initialize the XGBClassifier
xgb_full = XGBClassifier(random_state=42)

# Cross-validation
cv_scores = cross_val_score(xgb_full, X_train_full, y_train, cv=10, scoring='f1_weighted')

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the average cross-validation score
print("Average cross-validation score: ", cv_scores.mean())

Cross-validation scores:  [0.47578739 0.48318092 0.43678618 0.4457245  0.42455999 0.41683211
 0.42524841 0.48096528 0.44084305 0.46240597]
Average cross-validation score:  0.44923338044960726


In [19]:
# Positive correlations are shown by bars extending above the horizontal line (y=0), 
# indicating that an increase in these features is associated with higher values of 'result_match'.

# Negative correlations, shown by bars below the line, 
# suggest that an increase in these features is associated with lower values of 'result_match'.

In [20]:
# Calculate the Pearson correlation matrix
correlation_matrix = df.corr()

# Extract the correlations with the target variable 'result_match'
correlations_with_target = correlation_matrix['result_match'].sort_values(ascending=False)

correlations_with_target

# Select the top 5 positively correlated and top 5 negatively correlated features (excluding the target itself)
top_positive_correlated_features = correlations_with_target.index[1:6].tolist()
top_negative_correlated_features = correlations_with_target.index[-5:].tolist()
selected_features = top_positive_correlated_features + top_negative_correlated_features

# Create a new dataframe with the selected features and the target variable
selected_data = df[selected_features + ['result_match']]

# Create a heatmap for the selected features
plt.figure(figsize=(10, 8))
sns.heatmap(selected_data.corr(), annot=True, fmt=".2f", cmap='viridis')
plt.title('Heatmap of Selected Feature Correlations')
plt.show()

In [21]:
df.shape

(3040, 40)

In [32]:
df_ = df.copy()

In [33]:
# 'avg_possession_home', 'combined_3_alt', 'new_feature_4', 
# 'combined_1', 'diff_player_10', 'diff_player_7'
# 'avg_player_rating_away', 'combined_3_alt', 'new_feature_4'
# 'combined_2','goals_ratio_rolling', 'new_feature_4'
# 'avg_player_rating_away', 'avg_player_rating_home', 'combined_3_alt'


# 'combined_3_alt', 'new_feature_4'
# 'combined_1', 'combined_3_alt'
# 'combined_1', 'diff_player_10'
# 'combined_2', 'new_feature_4'
# 'combined_1', 'new_feature_4'
# 'new_feature_4', 'new_feature_5'
# 'combined_1', 'diff_player_7'
# 'avg_possession_home', 'diff_player_11'
# 'diff_player_10', 'diff_player_7'
# 'avg_possession_home', 'combined_3_alt'
# 'avg_possession_home', 'diff_player_8'

In [34]:
df_['ratio_away_rating_player_7_8_9_10_11'] = (df_['player_rating_away_player_7'] * df_['player_rating_away_player_8'] * df_['player_rating_away_player_9'] * df_['player_rating_away_player_10'] * df_['player_rating_away_player_11']) / 10000000
df_['ratio_home_rating_player_7_8_9_10_11'] = (df_['player_rating_away_player_7'] * df_['player_rating_away_player_8'] * df_['player_rating_away_player_9'] * df_['player_rating_home_player_10'] * df_['player_rating_home_player_11']) / 10000000

# diff_player_10|diff_player_7
# df_['combined_4'] = df_['diff_player_10'] * df_['diff_player_7']
# df_['combined_4_alt'] = df_['diff_player_10'] + df_['diff_player_7']

# Combination 1: new_feature_3 and new_feature_4
df_['combined_1'] = df_['new_feature_3'] * df_['new_feature_4']

# Combination 2: new_feature_4, new_feature_5, and avg_player_rating_away
df_['combined_2'] = df_['new_feature_4'] * df_['new_feature_5'] * df_['avg_player_rating_away']

# Combination 3: new_feature_3 with avg_player_rating_away
df_['combined_3_alt'] = df_['new_feature_3'] * df_['avg_player_rating_away']

In [35]:
# 'new_feature_3', 'avg_player_rating_away', 'new_feature_5', 'new_feature_4', 'avg_player_rating_away', 'new_feature_3',
# df_.drop([
#         'player_rating_away_player_7', 'player_rating_away_player_8', 'player_rating_away_player_9', 'player_rating_away_player_10', 'player_rating_away_player_11',
#          'player_rating_home_player_7', 'player_rating_home_player_8', 'player_rating_home_player_9', 'player_rating_home_player_10', 'player_rating_home_player_11'], axis=1, inplace=True)

In [36]:
# Creating new features based on the three-feature interactions
# df_['new_feature_99'] = (df_['avg_player_rating_away'] * df_['combined_3_alt']) / (df_['away_avg_goals_conceded'] + 1)

In [37]:
df_.shape

(3040, 45)

In [38]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

X = df_.drop('result_match', axis=1)
y = df_['result_match']
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [39]:
from sklearn.model_selection import cross_val_score

# Initialize the XGBClassifier
xgb_full = XGBClassifier(random_state=42)

xgb_full.fit(X_train_full, y_train)

# Cross-validation
cv_scores = cross_val_score(xgb_full, X_train_full, y_train, cv=5, scoring='f1_weighted')

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the average cross-validation score
print("Average cross-validation score: ", cv_scores.mean())

Cross-validation scores:  [0.44266075 0.45128523 0.4589103  0.43415681 0.47867201]
Average cross-validation score:  0.45313702144243273


In [42]:
# Calculate the Pearson correlation matrix
X_train_full['result_match'] = y_train

correlation_matrix = X_train_full.corr()

# Extract the correlations with the target variable 'result_match'
correlations_with_target = correlation_matrix['result_match'].sort_values(ascending=False)

correlations_with_target

# Select the top 5 positively correlated and top 5 negatively correlated features (excluding the target itself)
top_positive_correlated_features = correlations_with_target.index[1:6].tolist()
top_negative_correlated_features = correlations_with_target.index[-5:].tolist()
selected_features = top_positive_correlated_features + top_negative_correlated_features

# Create a new dataframe with the selected features and the target variable
selected_data = X_train_full[selected_features + ['result_match']]

# Create a heatmap for the selected features
plt.figure(figsize=(10, 8))
sns.heatmap(selected_data.corr(), annot=True, fmt=".2f", cmap='viridis')
plt.title('Heatmap of Selected Feature Correlations')
plt.show()

In [54]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3040 entries, 386 to 3016
Data columns (total 36 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   result_match                          3040 non-null   int64  
 1   home_avg_goals_scored                 3040 non-null   float64
 2   away_avg_goals_scored                 3040 non-null   float64
 3   home_avg_goals_conceded               3040 non-null   float64
 4   away_avg_goals_conceded               3040 non-null   float64
 5   home_streak_wins                      3040 non-null   int64  
 6   away_streak_wins                      3040 non-null   int64  
 7   home_sum_points                       3040 non-null   int64  
 8   away_sum_points                       3040 non-null   int64  
 9   points_diff                           3040 non-null   int64  
 10  win_eachother_home                    3040 non-null   int64  
 11  win_eachother_away  

In [55]:
df_.describe().round(2)

,result_match,home_avg_goals_scored,away_avg_goals_scored,home_avg_goals_conceded,away_avg_goals_conceded,home_streak_wins,away_streak_wins,home_sum_points,away_sum_points,points_diff,...,ratio_home_possession_rating,ratio_away_possession_rating,goals_ratio_rolling,shots_on_target_ratio_rolling,ratio_away_rating_player_7_8_9_10_11,ratio_home_rating_player_7_8_9_10_11,combined_1,combined_2,combined_3_alt,new_feature_99
count,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,...,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00,3040.00
mean,1.17,1.33,1.36,1.36,1.33,0.58,0.75,13.24,13.51,-0.27,...,37.48,37.58,0.76,0.81,269.93,266.17,106.99,268391.77,-49.17,-5823.84
std,0.84,0.53,0.55,0.49,0.48,1.18,1.21,5.74,5.70,7.66,...,4.42,4.34,0.32,0.27,74.16,53.19,1045.59,4553885.73,1041.48,40487.81
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-28.00,...,0.00,0.00,0.16,0.24,121.97,103.10,-3057.60,-32562216.96,-4472.00,-384592.00
25%,0.00,1.00,1.00,1.00,1.00,0.00,0.00,9.00,9.00,-5.00,...,36.25,36.29,0.53,0.61,215.27,227.45,-654.30,-433487.52,-730.80,-25990.12
50%,1.00,1.30,1.30,1.30,1.30,0.00,0.00,13.00,13.00,0.00,...,38.02,38.06,0.70,0.76,249.46,260.02,0.00,0.00,0.00,0.00
75%,2.00,1.60,1.70,1.70,1.60,1.00,1.00,17.00,18.00,5.00,...,39.63,39.67,0.91,0.96,319.24,298.99,736.20,479976.00,666.00,20073.70
max,2.00,4.50,4.20,4.00,4.00,11.00,11.00,30.00,30.00,25.00,...,47.30,47.84,2.76,2.13,469.31,448.37,4609.60,80676288.00,3156.20,200505.60


In [43]:
df_['home_sum_points']

386     16
412     28
423     22
441     19
464     11
        ..
2952    21
2966     5
2975    17
2999    20
3016     0
Name: home_sum_points, Length: 3040, dtype: int64

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, KernelPCA

preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", KernelPCA(n_components=2, random_state=42)),
    ]
)

clusterer = Pipeline(
   [
       (
           "kmeans",
           KMeans(
               n_clusters=3,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=42,
           ),
       ),
   ]
)

pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

In [35]:
pipe.fit(X_train_full)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('pca',
                                  KernelPCA(n_components=2,
                                            random_state=42))])),
                ('clusterer',
                 Pipeline(steps=[('kmeans',
                                  KMeans(max_iter=500, n_clusters=3, n_init=50,
                                         random_state=42))]))])

In [36]:
X_train_full.shape

(2736, 36)

In [37]:
import os

output_dir = "../data/transform/"
filename= "df_engineered.csv"

full_path = os.path.join(output_dir, filename)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_.to_csv(full_path, index=False)